In [3]:
import pickle
import pandas as pd

# Ruta al archivo del modelo
model_path = 'modelo/model.pkl'

# Cargar el modelo desde el archivo .pkl
def load_model(path):
    with open(path, 'rb') as file:
        model = pickle.load(file)
    return model

# Cargar el modelo
model = load_model(model_path)

# Ejemplo de datos de entrada
# Cambia esto según las características esperadas por tu modelo

age = int(input("Edad: "))
dependents= int(input("Dependientes: "))
occupation = input("Ocupación: ")
city_tier = input("Ciudad: ")
rent=   int(input("Renta (₹): "))
loan=   int(input("Préstamo (₹): "))
income = int(input("Ingreso mensual (₹): "))
disposible_income = int(input("Ingreso disponible (₹): "))
gasto_fijo = int(input("Gastos fijos (₹): "))
gasto_variable =    int(input("Gastos variables (₹): "))


# Hacer predicciones
predictions = model.predict(age,dependents,occupation,city_tier,rent,loan,income,disposible_income,gasto_fijo,gasto_variable)

# Mostrar resultados
print("Predicciones:", predictions)


FileNotFoundError: [Errno 2] No such file or directory: 'modelo.zip/model.pkl'